In [ ]:
### Personal Dataset Name: showcase
### https://www.kaggle.com/datasets/wilmerarltstrmberg/recipe-dataset-over-2m

######### STEP 1) Check Public dataset
SELECT * FROM `solution-play.showcase.recipe` LIMIT 1000

######### STEP 2) filter out to reduce size
CREATE OR REPLACE TABLE showcase.recipe_shorten
AS SELECT ` title` as title, ingredients, directions
FROM `solution-play.showcase.recipe` limit 20000;

######### STEP 3) Connection/모델 생성
CREATE OR REPLACE MODEL `showcase.text_embedding_model`
REMOTE WITH CONNECTION `solution-play.us.remote_model_connection`
OPTIONS(endpoint = 'text-embedding-preview-0409');

######### STEP 4) Generate Embedding
CREATE OR REPLACE TABLE `solution-play.showcase.embedding_for_recipe` AS (
SELECT *
FROM ML.GENERATE_EMBEDDING(
  MODEL `solution-play.showcase.text_embedding_model`,
  (SELECT ingredients as content, title, ingredients, directions from `solution-play.showcase.recipe_shorten`),
  STRUCT(TRUE AS flatten_json_output)
)WHERE ARRAY_LENGTH(ml_generate_embedding_result)=768);

######### STEP 5)  Generate Index
## https://cloud.google.com/bigquery/docs/vector-index?hl=ko
CREATE OR REPLACE VECTOR INDEX embeddings_index ON showcase.embedding_for_recipe(ml_generate_embedding_result)
OPTIONS(
  index_type = 'IVF',
  distance_type = 'COSINE',
  ivf_options = '{"num_lists":500}'
);

######### STEP 6) Information Schema to check the progress of the index
SELECT * FROM `solution-play.showcase.INFORMATION_SCHEMA.VECTOR_INDEX_COLUMNS`;

######### STEP 7) Semantic Search
SELECT query.query, base.title, base.content, base.directions
FROM VECTOR_SEARCH(
  TABLE `showcase.embedding_for_recipe`, 'ml_generate_embedding_result',
  (
  SELECT ml_generate_embedding_result, content AS query
  FROM ML.GENERATE_EMBEDDING(
  MODEL `solution-play.showcase.text_embedding_model`,
  (SELECT 'egg tart?' AS content))
  ),
  top_k => 5, options => '{"fraction_lists_to_search": 0.01}');